<a href="https://colab.research.google.com/github/vcautela/Decison_Tree/blob/main/Regress%C3%A3o_N%C3%A3o_Linear(DecisionTree).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libs

In [45]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import export_graphviz
import pydot


In [46]:
df = pd.read_excel('/content/temps.xlsx')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [47]:
df.describe()

,year,month,day,temp_2,temp_1,average,actual
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     348 non-null    int64  
 1   month    348 non-null    int64  
 2   day      348 non-null    int64  
 3   week     348 non-null    object 
 4   temp_2   348 non-null    int64  
 5   temp_1   348 non-null    int64  
 6   average  348 non-null    float64
 7   actual   348 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 21.9+ KB


## pre-processing (dummies)

In [49]:
df = pd.get_dummies(df)

In [50]:
df.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,0,0,0,0,0,1,0


In [51]:
labels = np.array(df['actual'])

features = df.drop('actual', axis=1)

feature_list = list(features.columns)

features = np.array(features)

In [52]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42)

In [53]:
baseline_preds = test_features[:, feature_list.index('average')]

baseline_error = abs(baseline_preds - test_labels)
print('Baseline error average', round(np.mean(baseline_error), 2))

Baseline error average 5.06


test_features[:, feature_list.index('average')]: Esta parte está utilizando indexação de arrays. test_features é uma matriz (ou array multidimensional), e o trecho [:, feature_list.index('average')] está selecionando todas as linhas (:) da coluna cujo índice é o resultado da expressão feature_list.index('average')

In [54]:
# n_estimator, quantas vezes irá rodar
rf = RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(train_features, train_labels);

In [55]:
prediction_rf =  rf.predict(test_features)

error_rf = abs(test_labels - prediction_rf)

r_sq = rf.score(features, labels)

print('R²:', r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, prediction_rf))
print('MSE:', metrics.mean_squared_error(test_labels, prediction_rf))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, prediction_rf)))

R²: 0.932094797587982
MAE: 3.932057471264368
MSE: 26.68358100000001
RMSE: 5.165615258611505


comparando o MAE com baseline error average, verificamos um modelo melhor que a nossa média de erro


## realizando pelo método Boosting

### ADA Boosting

In [56]:
ada = AdaBoostRegressor(n_estimators=100)
ada.fit(train_features, train_labels)

ada_pred = ada.predict(test_features)

In [57]:
error_ada = abs(test_labels - ada_pred)
r_sq = ada.score(features, labels)

print('R²:', r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, ada_pred))
print('MSE:', metrics.mean_squared_error(test_labels, ada_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, ada_pred)))

R²: 0.8782516169126122
MAE: 3.732001691495529
MSE: 24.914050286974234
RMSE: 4.991397628618084


### gradient boost regressor

In [58]:
gbr = GradientBoostingRegressor(n_estimators=100)
gbr.fit(train_features, train_labels)
gbr_pred = gbr.predict(test_features)

In [59]:
error_gbr = abs(test_labels - gbr_pred)
r_sq = gbr.score(features, labels)

print('R²:', r_sq)
print('MAE:', metrics.mean_absolute_error(test_labels, gbr_pred))
print('MSE:', metrics.mean_squared_error(test_labels, gbr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, gbr_pred)))


R²: 0.9175858974224234
MAE: 4.079508514057006
MSE: 28.51257825030832
RMSE: 5.339717057139668


In [60]:
# visualização de árvore de decisão

rf = RandomForestRegressor(max_depth=3)
rf.fit(train_features,train_labels)

tree = rf.estimators_[5]
tree
export_graphviz(tree, out_file='tree.dot', feature_names=feature_list, rounded= True, precision=1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')

### feature importances

É um método onde consigo avaliar quais são as variáveis mais relevantes para o meu modelo.

quais variáveis são usadas na minha árvore?

In [61]:
importances = list(rf.feature_importances_)

feature_importance = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

feature_importance = sorted(feature_importance, key= lambda x: x[1], reverse=True)

[print("Feature: {:20} importance {}".format(*pair)) for pair in feature_importance];

Feature: temp_1               importance 0.79
Feature: average              importance 0.19
Feature: month                importance 0.01
Feature: year                 importance 0.0
Feature: day                  importance 0.0
Feature: temp_2               importance 0.0
Feature: week_Fri             importance 0.0
Feature: week_Mon             importance 0.0
Feature: week_Sat             importance 0.0
Feature: week_Sun             importance 0.0
Feature: week_Thurs           importance 0.0
Feature: week_Tues            importance 0.0
Feature: week_Wed             importance 0.0


In [62]:
importances = list(ada.feature_importances_)

feature_importance = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

feature_importance = sorted(feature_importance, key= lambda x: x[1], reverse=True)

[print("Feature: {:20} importance {}".format(*pair)) for pair in feature_importance];

Feature: temp_1               importance 0.52
Feature: average              importance 0.28
Feature: temp_2               importance 0.08
Feature: month                importance 0.04
Feature: day                  importance 0.03
Feature: week_Mon             importance 0.03
Feature: week_Sun             importance 0.01
Feature: year                 importance 0.0
Feature: week_Fri             importance 0.0
Feature: week_Sat             importance 0.0
Feature: week_Thurs           importance 0.0
Feature: week_Tues            importance 0.0
Feature: week_Wed             importance 0.0


In [63]:
importances = list(gbr.feature_importances_)

feature_importance = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

feature_importance = sorted(feature_importance, key= lambda x: x[1], reverse=True)

[print("Feature: {:20} importance {}".format(*pair)) for pair in feature_importance];

Feature: temp_1               importance 0.63
Feature: average              importance 0.3
Feature: day                  importance 0.02
Feature: month                importance 0.01
Feature: temp_2               importance 0.01
Feature: week_Fri             importance 0.01
Feature: year                 importance 0.0
Feature: week_Mon             importance 0.0
Feature: week_Sat             importance 0.0
Feature: week_Sun             importance 0.0
Feature: week_Thurs           importance 0.0
Feature: week_Tues            importance 0.0
Feature: week_Wed             importance 0.0
